# 07 compare the different variations run in 05a
- alternative reference regions and/or changes in subject inclusion

versions needed to compare:

Data versions:
original reference region + original subject set
Paracentral-gm reference region + original subject set
Paracentral-gm reference region + new subject set
RAS-ref1 reference region + original subject set
RAS-ref1 reference region + new subject set

In [1]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import numpy as np;
import pandas as pd;
from sklearn import preprocessing 
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import pickle



C:\Users\Lauren.Koenig\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# read in and clean data

In [2]:
data_folder_loc = "//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/"
code_folder_loc = "//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/Riluzole FDG/"

In [3]:
x_var_list = ['Ab42_40', 'GFAP', 'NFL', 'pTau181', 'pTau217', 'pTau231']
y_var_list = ['RASref1_gm', 'Paracentral_gm', 'Ab42_log10', 'pTau181_Ab42_log10', 'Ab40_log10']

rename_columns = ['model_p_val_baseline', 'model_B_val_baseline',
'model_n_baseline', 
       'model_p_val_Intercept', 'model_B_val_Intercept',
       'model_p_val_x_var', 'model_B_val_x_var',
        'model_p_val_age','model_B_val_age',
       'model_p_val_Education_years', 'model_B_val_Education_years', 
       'model_p_val_sex[T.M]', 'model_B_val_sex[T.M]',
        'model_p_val_apoe4_carrier[T.Yes]', 'model_B_val_apoe4_carrier[T.Yes]',
         'model_p_val_race[T.White/NH]', 'model_B_val_race[T.White/NH]']

covariate_pval_columns = [ 'model_p_val_age',
       'model_p_val_Education_years', 
       'model_p_val_sex[T.M]', 
        'model_p_val_apoe4_carrier[T.Yes]',
         'model_p_val_race[T.White/NH]']


# define biomarker groups
FDG_columns = ['Avg_MedOrbFrontal', 'Graycer_gm', 'Avg_PCC', 'Avg_Hip', 'MTL_gm', 'CO', 'Temp', 'SensMot_gm', 'L_Hip',
 'Vermis_gm', 'FRONTAL_gm', 'AC_gm', 'Precun_gm', 'Par_gm', 'RASref1_gm', 'Temp_gm', 'Paracentral_gm', 'R_Hip', 'PostCing_gm']

plasma_columns = [ 'Ab40', 'Ab42', 'Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau181_Ab42']
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

cog_columns = ['MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']
vol_columns = ['VOL_Precun_Lz', 'VOL_Precun_Rz', 'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz', 'VOL_Inf_Mid_Fus_Temp_Lz', 
'VOL_Inf_Mid_Fus_Temp_Rz','VOL_MidFront_Lz', 'VOL_MidFront_Rz']
      

In [4]:
# read in data
results_new_para, temp, data4 = pd.read_pickle(code_folder_loc + '/output/05a_data_new_para_rr.pkl')
results_new_para = results_new_para[results_new_para['x_var'].isin(x_var_list) & ~results_new_para['y_var'].isin(y_var_list)]
results_new_para = results_new_para.rename(columns = dict(zip(rename_columns, 
['new_para_' + ele for ele in rename_columns])))

# read in data
results_pons, temp, data2 = pd.read_pickle(code_folder_loc + '/output/05a_data_pons_rr.pkl')
results_pons = results_pons[results_pons['x_var'].isin(x_var_list) & ~results_pons['y_var'].isin(y_var_list)]
results_pons = results_pons.rename(columns = dict(zip(rename_columns, 
['pons_' + ele for ele in rename_columns])))

'''
# read in data
results_composite, temp, data1 = pd.read_pickle(code_folder_loc + '/output/05a_data_composite_rr.pkl')
results_composite = results_composite[results_composite['x_var'].isin(x_var_list) & ~results_composite['y_var'].isin(y_var_list)]
results_composite = results_composite.rename(columns = dict(zip(rename_columns, 
['composite_' + ele for ele in rename_columns])))

# read in data
results_para, temp, data3 = pd.read_pickle(code_folder_loc + '/output/05a_data_para_rr.pkl')
results_para = results_para[results_para['x_var'].isin(x_var_list) & ~results_para['y_var'].isin(y_var_list)]
results_para = results_para.rename(columns = dict(zip(rename_columns, 
['para_' + ele for ele in rename_columns])))

# read in data
results_ras, temp, data5 = pd.read_pickle(code_folder_loc + '/output/05a_data_ras_rr.pkl')
results_ras = results_ras[results_ras['x_var'].isin(x_var_list) & ~results_ras['y_var'].isin(y_var_list)]
results_ras = results_ras.rename(columns = dict(zip(rename_columns, 
['ras_' + ele for ele in rename_columns])))

# read in data
results_subset_new_para, temp, data5_subset = pd.read_pickle(code_folder_loc + '/output/05a_data_new_para_rr_subset.pkl')
results_subset_new_para = results_subset_new_para[results_subset_new_para['x_var'].isin(x_var_list) & ~results_subset_new_para['y_var'].isin(y_var_list)]
results_subset_new_para = results_subset_new_para.rename(columns = dict(zip(rename_columns, 
['subset_new_para_' + ele for ele in rename_columns])))

# read in data
results_orig_new_para, temp, data6 = pd.read_pickle(code_folder_loc + '/output/with_RIL8_included/05a_data_new_para_rr.pkl')
results_orig_new_para = results_orig_new_para[results_orig_new_para['x_var'].isin(x_var_list) & ~results_orig_new_para['y_var'].isin(y_var_list)]
results_orig_new_para = results_orig_new_para.rename(columns = dict(zip(rename_columns, 
['orig_new_para_' + ele for ele in rename_columns])))
'''

"\n# read in data\nresults_composite, temp, data1 = pd.read_pickle(code_folder_loc + '/output/05a_data_composite_rr.pkl')\nresults_composite = results_composite[results_composite['x_var'].isin(x_var_list) & ~results_composite['y_var'].isin(y_var_list)]\nresults_composite = results_composite.rename(columns = dict(zip(rename_columns, \n['composite_' + ele for ele in rename_columns])))\n\n# read in data\nresults_para, temp, data3 = pd.read_pickle(code_folder_loc + '/output/05a_data_para_rr.pkl')\nresults_para = results_para[results_para['x_var'].isin(x_var_list) & ~results_para['y_var'].isin(y_var_list)]\nresults_para = results_para.rename(columns = dict(zip(rename_columns, \n['para_' + ele for ele in rename_columns])))\n\n# read in data\nresults_ras, temp, data5 = pd.read_pickle(code_folder_loc + '/output/05a_data_ras_rr.pkl')\nresults_ras = results_ras[results_ras['x_var'].isin(x_var_list) & ~results_ras['y_var'].isin(y_var_list)]\nresults_ras = results_ras.rename(columns = dict(zip(ren

# Pons vs New Para

In [5]:
comp_new_para = pd.merge(results_pons, results_new_para, how = 'inner')
comp_new_para

,y_var,x_var,pons_model_p_val_baseline,pons_model_B_val_baseline,pons_model_n_baseline,pons_model_p_val_Intercept,pons_model_p_val_x_var,pons_model_B_val_Intercept,pons_model_B_val_x_var,pons_model_p_val_sex[T.M],...,new_para_model_B_val_Education_years,new_para_model_B_val_x_var,new_para_model_p_val_age,new_para_model_B_val_age,new_para_model_p_val_sex[T.M],new_para_model_B_val_sex[T.M],new_para_model_p_val_apoe4_carrier[T.Yes],new_para_model_B_val_apoe4_carrier[T.Yes],new_para_model_p_val_race[T.White/NH],new_para_model_B_val_race[T.White/NH]
0,MTL_gm,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.5...,Intercept 1.274154e-15 Ab42_40_log10 ...,31,1.000000,0.538983,1.274154e-15,0.114689,NaN,...,0.467310,0.389317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AC_gm,Ab42_40,Intercept 0.024900 sex[T.M] 0.0...,Intercept 0.464529 sex[T.M] -1.2...,31,0.024900,0.655576,4.645291e-01,-0.072157,0.001059,...,NaN,0.038558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Precun_gm,Ab42_40,Intercept 0.100797 sex[T.M] 0.0...,Intercept 0.377718 sex[T.M] -0.9...,31,0.100797,0.269296,3.777183e-01,0.204866,0.014219,...,0.364864,0.373903,0.007766,0.447645,NaN,NaN,NaN,NaN,NaN,NaN
3,Par_gm,Ab42_40,Intercept 1.000000 Ab42_40_log10 0.9...,Intercept -1.179612e-16 Ab42_40_log10 ...,31,1.000000,0.901895,-1.179612e-16,-0.023085,NaN,...,NaN,0.192024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Temp_gm,Ab42_40,Intercept 0.106899 sex[T.M] 0.0...,Intercept 0.372288 sex[T.M] -0.9...,31,0.106899,0.390890,3.722882e-01,0.159058,0.015902,...,NaN,0.293636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,VOL_Precun_InfPar_Supramarg_Rz,pTau231,Intercept 1.000000 age ...,Intercept -9.714451e-17 age ...,32,1.000000,0.008586,-9.714451e-17,0.390247,NaN,...,0.355296,0.390247,0.000075,0.642682,NaN,NaN,NaN,NaN,NaN,NaN
614,VOL_LatOccLingCun_Lz,pTau231,Intercept 1.000000 pTau231_log10 0.0...,Intercept -1.665335e-16 pTau231_log10 ...,32,1.000000,0.040319,-1.665335e-16,0.364402,NaN,...,NaN,0.364402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,VOL_LatOccLingCun_Rz,pTau231,Intercept 1.000000 pTau231_log10 0.0...,Intercept 0.000000 pTau231_log10 0.4...,32,1.000000,0.012228,0.000000e+00,0.437723,NaN,...,NaN,0.437723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,VOL_InfParSupra_Lz,pTau231,Intercept 1.000000 age 0.0...,Intercept -2.775558e-17 age ...,32,1.000000,0.071284,-2.775558e-17,0.249357,NaN,...,NaN,0.249357,0.000019,0.678954,NaN,NaN,NaN,NaN,NaN,NaN


## x_var significance

In [6]:
comp_new_para.loc[(comp_new_para['pons_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 
'pons_model_p_val_x_var', 'new_para_model_p_val_x_var',
'pons_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'pons_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,pons_model_p_val_x_var,new_para_model_p_val_x_var,pons_model_B_val_x_var,new_para_model_B_val_x_var,pons_model_n_baseline,new_para_model_n_baseline


In [7]:
comp_new_para.loc[(comp_new_para['pons_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), 
['x_var', 'y_var',
'pons_model_p_val_x_var', 'new_para_model_p_val_x_var',
'pons_model_B_val_x_var', 'new_para_model_B_val_x_var', 
'pons_model_n_baseline','new_para_model_n_baseline']]

,x_var,y_var,pons_model_p_val_x_var,new_para_model_p_val_x_var,pons_model_B_val_x_var,new_para_model_B_val_x_var,pons_model_n_baseline,new_para_model_n_baseline
0,Ab42_40,MTL_gm,0.538983,0.010588,0.114689,0.389317,31,35
2,Ab42_40,Precun_gm,0.269296,0.021403,0.204866,0.373903,31,35
18,pTau181,MTL_gm,0.103894,0.000048,0.271625,0.547130,37,41
23,pTau181,PostCing_gm,0.281572,0.015868,0.159634,0.373023,37,41
26,pTau217,Precun_gm,0.240472,0.044767,-0.206857,-0.327454,34,38
30,pTau231,MTL_gm,0.164072,0.032529,0.247989,0.347523,33,38


In [8]:
(comp_new_para['pons_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

count    618.000
mean       0.011
std        0.095
min       -0.598
25%       -0.000
50%        0.000
75%        0.000
max        0.924
dtype: float64

In [9]:
comp_new_para.loc[(abs(comp_new_para['pons_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'pons_model_p_val_x_var',
       'pons_model_B_val_x_var', 'pons_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

,x_var,y_var,pons_model_p_val_x_var,pons_model_B_val_x_var,pons_model_n_baseline,new_para_model_p_val_x_var,new_para_model_B_val_x_var,new_para_model_n_baseline
18,pTau181,MTL_gm,0.103894,0.271625,37,0.000048,0.547130,41
0,Ab42_40,MTL_gm,0.538983,0.114689,31,0.010588,0.389317,35
23,pTau181,PostCing_gm,0.281572,0.159634,37,0.015868,0.373023,41
2,Ab42_40,Precun_gm,0.269296,0.204866,31,0.021403,0.373903,35
30,pTau231,MTL_gm,0.164072,0.247989,33,0.032529,0.347523,38
26,pTau217,Precun_gm,0.240472,-0.206857,34,0.044767,-0.327454,38
10,GFAP,Temp_gm,0.632654,-0.079818,37,0.061341,-0.291202,42
9,GFAP,Par_gm,0.997122,0.000614,37,0.073024,-0.271715,41
4,Ab42_40,Temp_gm,0.390890,0.159058,31,0.082178,0.293636,36
32,pTau231,Precun_gm,0.569832,0.096389,33,0.112292,0.237984,37


## covariate significance

In [10]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['pons_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

,new_para_model_p_val_Education_years,new_para_model_p_val_age,pons_model_p_val_sex[T.M],x_var,y_var
20,0.006586,0.016969,NaN,pTau181,Precun_gm
0,0.002342,NaN,NaN,Ab42_40,MTL_gm
6,0.005543,NaN,NaN,GFAP,MTL_gm
12,0.001880,NaN,NaN,NFL,MTL_gm
18,0.000017,NaN,NaN,pTau181,MTL_gm
24,0.013116,NaN,NaN,pTau217,MTL_gm
9,NaN,0.028485,NaN,GFAP,Par_gm
15,NaN,0.036415,NaN,NFL,Par_gm
21,NaN,0.041938,NaN,pTau181,Par_gm
1,NaN,NaN,0.001059,Ab42_40,AC_gm


# Composite vs New Para

In [ ]:
comp_new_para = pd.merge(results_composite, results_new_para, how = 'inner')
comp_new_para

## x_var significance

In [ ]:
comp_new_para.loc[(comp_new_para['composite_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 'composite_model_p_val_x_var',
       'composite_model_B_val_x_var', 'composite_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
comp_new_para.loc[(comp_new_para['composite_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), ['x_var', 'y_var', 'composite_model_p_val_x_var',
       'composite_model_B_val_x_var', 'composite_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
(comp_new_para['composite_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

In [ ]:
comp_new_para.loc[(abs(comp_new_para['composite_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'composite_model_p_val_x_var',
       'composite_model_B_val_x_var', 'composite_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['composite_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp = temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# new_para vs subset new_para

In [ ]:
new_para_new_para = pd.merge(results_new_para, results_subset_new_para, how = 'inner')
new_para_new_para

## x_var significance

In [ ]:
new_para_new_para[(new_para_new_para['new_para_model_p_val_x_var'] < 0.05) & (new_para_new_para['subset_new_para_model_p_val_x_var'] >= 0.05)]

In [ ]:
new_para_new_para[(new_para_new_para['new_para_model_p_val_x_var'] >= 0.05) & (new_para_new_para['subset_new_para_model_p_val_x_var'] < 0.05)]

In [ ]:
(new_para_new_para['new_para_model_p_val_x_var']  - new_para_new_para['subset_new_para_model_p_val_x_var']).describe().round(3)

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = new_para_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = new_para_new_para[['subset_new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = new_para_new_para[a_sig != b_sig].drop(columns = new_para_new_para.columns[new_para_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# Ras vs new para

In [ ]:
ras_new_para = pd.merge(results_ras, results_new_para, how = 'inner')
ras_new_para

## x_var significance

In [ ]:
ras_new_para[(ras_new_para['ras_model_p_val_x_var'] < 0.05) & (ras_new_para['new_para_model_p_val_x_var'] >= 0.05)]

In [ ]:
ras_new_para[(ras_new_para['ras_model_p_val_x_var'] >= 0.05) & (ras_new_para['new_para_model_p_val_x_var'] < 0.05)]

In [ ]:
(ras_new_para['ras_model_p_val_x_var']  - ras_new_para['new_para_model_p_val_x_var']).describe().round(3)

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = ras_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = ras_new_para[['ras_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = ras_new_para[a_sig != b_sig].drop(columns = ras_new_para.columns[ras_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# Composite vs Old Para

In [ ]:
comp_para = pd.merge(results_composite, results_para, how = 'inner')
comp_para

## x_var significance

In [ ]:
comp_para[(comp_para['composite_model_p_val_x_var'] < 0.05) & (comp_para['para_model_p_val_x_var'] >= 0.05)]

In [ ]:
comp_para[(comp_para['composite_model_p_val_x_var'] >= 0.05) & (comp_para['para_model_p_val_x_var'] < 0.05)]

In [ ]:
(comp_para['composite_model_p_val_x_var']  - comp_para['para_model_p_val_x_var']).describe().round(3)

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = comp_para[['composite_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_para[['para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_para[a_sig != b_sig].drop(columns = comp_para.columns[comp_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp

# New Para with and without RIL8

In [ ]:
comp_new_para = pd.merge(results_orig_new_para, results_new_para, how = 'inner')
comp_new_para

## x_var significance

In [ ]:
comp_new_para.loc[(comp_new_para['orig_new_para_model_p_val_x_var'] < 0.05) & (comp_new_para['new_para_model_p_val_x_var'] >= 0.05), ['x_var', 'y_var', 'orig_new_para_model_p_val_x_var',
       'orig_new_para_model_B_val_x_var', 'orig_new_para_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
comp_new_para.loc[(comp_new_para['orig_new_para_model_p_val_x_var'] >= 0.05) & (comp_new_para['new_para_model_p_val_x_var'] < 0.05), ['x_var', 'y_var', 'orig_new_para_model_p_val_x_var',
       'orig_new_para_model_B_val_x_var', 'orig_new_para_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']]

In [ ]:
(comp_new_para['orig_new_para_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var']).describe().round(3)

In [ ]:
comp_new_para.loc[(abs(comp_new_para['orig_new_para_model_p_val_x_var']  - comp_new_para['new_para_model_p_val_x_var'])> 0.1), ['x_var', 'y_var', 'orig_new_para_model_p_val_x_var',
       'orig_new_para_model_B_val_x_var', 'orig_new_para_model_n_baseline', 'new_para_model_p_val_x_var', 'new_para_model_B_val_x_var',
       'new_para_model_n_baseline']].sort_values('new_para_model_p_val_x_var')

## covariate significance

In [ ]:
# those where covariate significance differs between a and b
a_sig = comp_new_para[['orig_new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)
b_sig = comp_new_para[['new_para_' + ele for ele in covariate_pval_columns]].notna().any(axis =1)

temp = comp_new_para[a_sig != b_sig].drop(columns = comp_new_para.columns[comp_new_para.columns.str.contains('B_val|Intercept|baseline|type|p_val_x_var')]).dropna(axis = 1, how = 'all')
temp = temp[temp.columns.sort_values()]
temp.loc[temp.isna().sort_values(temp.columns.to_list()).index, :]
temp